<a href="https://colab.research.google.com/github/aliferds/bcc325_prova02/blob/main/Prova02_BCC325___Analise_de_sentimentos_em_avaliacoes_de_produtos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Analise de Sentimento em avaliações de produtos #


## Importação do dataset ##

In [ ]:
!gdown '1JxPS6PHxsWxlo1Nc8Jay53EYyHq8nhuL' -O amazon.csv

Downloading...
From: https://drive.google.com/uc?id=1JxPS6PHxsWxlo1Nc8Jay53EYyHq8nhuL
To: /content/amazon.csv
100% 4.74M/4.74M [00:00<00:00, 132MB/s]


### -------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Instalação das bibliotecas

In [ ]:
!pip install pandas

## Importação das bibliotecas

In [ ]:
import pandas as pd

Abrindo o arquivo com pandas

In [ ]:
df = pd.read_csv("amazon.csv", on_bad_lines='skip')

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
# df.head(4) # Retorna os n valores iniciais

Verificando as colunas e os tipos de dados usados

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   product_id           1465 non-null   object
 1   product_name         1465 non-null   object
 2   category             1465 non-null   object
 3   discounted_price     1465 non-null   object
 4   actual_price         1465 non-null   object
 5   discount_percentage  1465 non-null   object
 6   rating               1465 non-null   object
 7   rating_count         1463 non-null   object
 8   about_product        1465 non-null   object
 9   user_id              1465 non-null   object
 10  user_name            1465 non-null   object
 11  review_id            1465 non-null   object
 12  review_title         1465 non-null   object
 13  review_content       1465 non-null   object
 14  img_link             1465 non-null   object
 15  product_link         1465 non-null   object
dtypes: obj

Verificando se há valores nulos

In [ ]:
# df.isnull().sum() # retorna a quantidade de valores nulos de acordo com a coluna

Convertendo os valores para float

In [ ]:
df['rating'].value_counts()

,count
rating,
4.1,244
4.3,230
4.2,228
4.0,129
3.9,123
4.4,123
3.8,86
4.5,75
4,52


## Pré Processamento
É definido a função para criar uma coluna, caso seja necessário

As colunas de review_title e review_content são concatenadas e adicionadas em uma nova coluna denominada review

Depois é removido todas as colunas que estiverem no vetor cols_to_drop

Finalmente é aplicado um filtro para não inserir a linha onde o rating está com o caractere '|' no lugar de um numero.

In [ ]:
def criar_coluna(tabela, nova_coluna, valor):
  tabela[nova_coluna] = valor

In [ ]:
criar_coluna(df, 'review', df['review_title'].str.cat(df['review_content'], sep=' '))

cols_to_drop = ['img_link', 'product_link', 'user_id',
                'user_name', 'product_id', 'review_id',
                'product_name', 'review_title', 'review_content']
df = df.drop(columns=[col for col in cols_to_drop if col in df.columns], axis=1)
df = df[df['rating'] != '|']
# df['rating'].value_counts()

## Convertendo os valores
Os valores de preço, desconto, porcentagem de desconto, contador de avaliações e avaliações são transformados em float

In [ ]:
def value_formatter(value):
  return value.str.replace(",",'').str.replace("₹",'').astype('float64')

df['discounted_price'] = value_formatter(df['discounted_price'])
df['actual_price'] = value_formatter(df['actual_price'])

df['rating_count'] = df['rating_count'].str.replace(',', '').astype('float64')

df['rating'] = df['rating'].astype('float64')

df['discount_percentage'] = df['discount_percentage'].str.replace('%','').astype('float64')
df['discount_percentage'] = df['discount_percentage'] / 100

df = df.dropna() # remove valores nulos
df = df.drop_duplicates() # remove valores duplicados

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1371 entries, 0 to 1464
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   category             1371 non-null   object 
 1   discounted_price     1371 non-null   float64
 2   actual_price         1371 non-null   float64
 3   discount_percentage  1371 non-null   float64
 4   rating               1371 non-null   float64
 5   rating_count         1371 non-null   float64
 6   about_product        1371 non-null   object 
 7   review               1371 non-null   object 
dtypes: float64(5), object(3)
memory usage: 96.4+ KB


## Verificando as palavras mais comuns

O código mostra as palavras mais comuns utilizando a função most_common_words

In [ ]:
from collections import Counter
import re

def words_unique_counter(words):
  all_text = ' '.join(words)
  words = re.findall(r'\b\w+\b', all_text.lower())
  unique_words = set(words)
  return len(unique_words)

def most_common_words(str_value, n, exclude_words=None):
  if exclude_words is None:
    exclude_words = []

  all_text = ' '.join(str_value)
  words = re.findall(r'\b\w+\b', all_text.lower())
  filtered_words = [word for word in words if word not in exclude_words]
  word_counts = Counter(filtered_words)
  most_common_words = [word for word, _ in word_counts.most_common(n)]
  return most_common_words


exclude_list = ['the', 'and', 'it', 'to', 'is', 'a', 'in', 'this', 'on', 'i',
                'with', 'for', 's', 'of', 't', 'so', 'at', 'as', 'that', 'you',
                'my', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0']

review = most_common_words(df['review'], 50, exclude_words=exclude_list)
about_product = most_common_words(df['about_product'], 50, exclude_words=exclude_list)
category = most_common_words(df['category'], 20)


categoria = words_unique_counter(df['category'])
print(f'Quantidade de palavras unicas em category: {categoria}')
sobre = words_unique_counter(df['about_product'])
print(f'Quantidade de palavras unicas em about_product: {sobre}')
analise = words_unique_counter(df['review'])
print(f'Quantidade de palavras unicas em review: {analise}')

print(f"Palavrras comuns em review: {review}")
print(f"Palavras mais comuns em about_product: {about_product}")
print(f"Palavras mais comuns em category: {category}")

Quantidade de palavras unicas em category: 370
Quantidade de palavras unicas em about_product: 9187
Quantidade de palavras unicas em review: 14831
Palavrras comuns em review: ['good', 'product', 'but', 'not', 'very', 'quality', 'have', 'use', 'nice', 'are', 'price', 'be', 'one', 'can', 'if', 'was', 'cable', 'like', 'from', 'money', 'no', 'best', 'or', 'easy', 'also', 'its', 'will', 'all', 'only', 'after', 'working', 'which', 'phone', 'charging', 'using', 'value', 'has', 'buy', 'great', 'well', 'tv', 'just', 'sound', 'time', 'battery', 'more', 'works', 'watch', 'amazon', 'when']
Palavras mais comuns em about_product: ['your', 'usb', 'or', 'can', 'cable', 'from', 'warranty', 'up', 'easy', 'c', 'power', 'charging', 'type', 'high', 'use', 'devices', 'design', 'be', 'not', 'water', 'all', 'compatible', 'product', 'by', '10', 'speed', 'any', 'charge', 'x', 'time', 'fast', 'display', 'free', 'more', 'will', 'which', 'quality', 'year', 'has', 'tv', 'watch', 'battery', 'an', 'are', 'features', 

Possiveis palavras relevantes em review:

'good' | 'nice' | 'great'

'bad' | 'terrible' | 'inferior'

'but' | 'only' | 'yet'

'not' | 'no'

'easy' | 'simple' | 'smooth'

'hard' | 'tricky' | 'difficult'

'defective' | 'faulty'

Possiveis palavras relevantes em about:

'fast'

'speed'

'free'

'quality'

'warranty'

In [ ]:
def mark_word_presence(texts, words_to_check):
  marked_reviews_count = []
  for text in texts:
    text_words = set(re.findall(r'\b\w+\b', text.lower()))
    count = sum(1 if word in text_words else 0 for word in words_to_check)
    marked_reviews_count.append(count)

  return marked_reviews_count

In [ ]:
review_good_words = ['good', 'nice', 'great']

nice_review = mark_word_presence(df['review'], review_good_words)

print(nice_review)

[1, 1, 2, 2, 3, 2, 2, 2, 2, 1, 1, 2, 1, 3, 2, 2, 3, 1, 2, 2, 1, 1, 1, 2, 3, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2, 2, 3, 2, 2, 1, 2, 3, 1, 2, 2, 1, 2, 1, 3, 2, 2, 2, 1, 3, 2, 1, 2, 3, 2, 2, 2, 3, 1, 1, 1, 1, 2, 3, 2, 3, 2, 1, 1, 3, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 2, 3, 3, 1, 1, 1, 3, 2, 3, 1, 3, 1, 3, 1, 1, 3, 2, 2, 3, 2, 3, 1, 2, 2, 2, 3, 3, 3, 2, 2, 1, 1, 1, 1, 2, 1, 2, 2, 3, 1, 2, 2, 2, 3, 2, 3, 3, 2, 3, 2, 3, 2, 1, 2, 3, 2, 2, 1, 2, 1, 1, 3, 2, 2, 2, 3, 2, 1, 2, 2, 0, 1, 3, 1, 1, 2, 2, 3, 2, 2, 2, 2, 1, 3, 3, 2, 3, 2, 2, 2, 2, 2, 2, 2, 3, 1, 1, 2, 3, 3, 3, 3, 3, 2, 2, 3, 2, 3, 2, 2, 2, 2, 3, 1, 1, 2, 1, 2, 1, 1, 3, 2, 3, 1, 2, 2, 1, 3, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 3, 3, 3, 1, 2, 3, 2, 2, 2, 1, 1, 2, 3, 2, 3, 2, 1, 2, 2, 2, 1, 2, 1, 2, 3, 2, 1, 1, 2, 3, 2, 3, 2, 2, 1, 3, 2, 1, 1, 3, 3, 1, 2, 2, 1, 3, 3, 1, 2, 2, 1, 3, 2, 3, 2, 1, 3, 3, 1, 2, 2, 2, 2, 3, 3, 2, 2, 3, 1, 2, 3, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 1, 1, 3, 2, 3, 2, 2, 3, 1, 1, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2, 3, 2, 2, 3, 2, 2, 2, 

In [ ]:
categoria1 = df['category'].head(1)
print(f'Categoria 1 linha {categoria1}')

Categoria 1 linha 0    Computers&Accessories|Accessories&Peripherals|...
Name: category, dtype: object
